In [1]:
from datetime import date

In [10]:
#valor tecnico

from datetime import date

# --- DATOS DE ENTRADA ---
# La fecha "hoy" la fijo al 13 de Septiembre de 2025 para replicar tu ejemplo
hoy = date(2025, 9, 27)
precio_bono = 1490  # Precio "sucio" o total que pagás por cada 100 V$N

# Datos de las condiciones de emisión
cer_inicial = 22.54   # CER 10 días hábiles antes de emisión (21/08/2020)
tasa_anual = 0.02     # 2.00%
capital_residual_Vn = 60 # Ya pagó 2 cuotas de 20% (quedan 3)

# Datos de mercado / contexto
cer_hoy = 632.59

# --- CÁLCULOS ---

# 1. Calcular el ajuste CER acumulado
ajuste_cer = cer_hoy / cer_inicial

# 2. Calcular los días corridos desde el último pago de cupón
fecha_ultimo_cupon = date(2025, 5, 9)
dias_corridos = (hoy - fecha_ultimo_cupon).days

# 3. Calcular los intereses corridos en VALOR NOMINAL (V$N)
# La tasa es 1% semestral (0.01) y el semestre tiene 180 días (base 30/360)
intereses_corr_Vn = capital_residual_Vn * (tasa_anual / 2) * (dias_corridos / 180)

# 4. Calcular el Valor Técnico en PESOS ($)
# Sumamos capital e intereses en V$N y LUEGO ajustamos por CER
valor_tecnico = (capital_residual_Vn + intereses_corr_Vn) * ajuste_cer

# 5. Calcular la Paridad
paridad = (precio_bono / valor_tecnico) * 100 # Se suele expresar en %

# --- RESULTADOS ---
print(f"Días corridos: {dias_corridos}")
print(f"Intereses corridos (en V$N): {intereses_corr_Vn:.4f}")
print(f"Valor Técnico ($): {valor_tecnico:.2f}")
print(f"Paridad: {paridad:.2f}%")

Días corridos: 141
Intereses corridos (en V$N): 0.4700
Valor Técnico ($): 1697.10
Paridad: 87.80%


In [13]:


# --- DATOS DE ENTRADA ---
precio_bono = precio_bono  # Este es el Precio Sucio (Dirty Price) según la convención argentina
cer_inicial = 22.54
cer_hoy = cer_hoy
capital_restante_nominal = 60  # El 60% del VNO de 100
tasa_anual = 0.02

# Fechas clave
hoy = date(2025, 9, 26) # Fijamos la fecha para que el cálculo sea consistente
ultimo_pago_cupon = date(2025, 5, 9) # Fecha correcta del último pago
proximo_pago_cupon = date(2025, 11, 9)

# --- PASO 1: CALCULAR EL VALOR RESIDUAL AJUSTADO (VRA) ---
# Es el capital que queda, ajustado por la inflación (CER) hasta hoy.
ajuste_cer = cer_hoy / cer_inicial
valor_residual_ajustado = capital_restante_nominal * ajuste_cer

print(f"Coeficiente de Ajuste CER: {ajuste_cer:.4f}")
print(f"Valor Residual Ajustado (VRA): ${valor_residual_ajustado:.2f}")

# --- PASO 2: CALCULAR LOS INTERESES CORRIDOS (IC) EN PESOS ---
# Se calculan sobre el VRA.
dias_corridos = (hoy - ultimo_pago_cupon).days
dias_totales_semestre = (proximo_pago_cupon - ultimo_pago_cupon).days
tasa_semestral = tasa_anual / 2

# Fórmula: IC = VRA * Tasa Semestral * (Días transcurridos / Días totales del período)
intereses_corridos_pesos = valor_residual_ajustado * tasa_semestral * (dias_corridos / dias_totales_semestre)

print(f"Días corridos desde el último pago: {dias_corridos}")
print(f"Días totales en el semestre: {dias_totales_semestre}")
print(f"Intereses Corridos (IC) en Pesos: ${intereses_corridos_pesos:.2f}")

# --- PASO 3: CALCULAR EL VALOR TÉCNICO (VT) ---
# Es la suma del capital adeudado más los intereses devengados, todo en pesos de hoy.
valor_tecnico = valor_residual_ajustado + intereses_corridos_pesos

print(f"-------------------------------------------")
print(f"VALOR TÉCNICO (VT) CALCULADO: ${valor_tecnico:.2f}")
print(f"-------------------------------------------")

# --- PASO 4: CALCULAR LA PARIDAD ---
# Compara el precio que se paga en el mercado con el valor contable (VT).
paridad = (precio_bono / valor_tecnico) * 100

print(f"Precio del Bono (Sucio): ${precio_bono:.2f}")
print(f"PARIDAD CALCULADA: {paridad:.2f}%")

Coeficiente de Ajuste CER: 28.0652
Valor Residual Ajustado (VRA): $1683.91
Días corridos desde el último pago: 140
Días totales en el semestre: 184
Intereses Corridos (IC) en Pesos: $12.81
-------------------------------------------
VALOR TÉCNICO (VT) CALCULADO: $1696.73
-------------------------------------------
Precio del Bono (Sucio): $1490.00
PARIDAD CALCULADA: 87.82%


In [14]:
from datetime import date
import pyxirr # <-- ¡Librería correcta!

# --- 1. DATOS DE ENTRADA ---
# Datos del mercado y del bono que ya conocemos.
precio_bono_sucio = 1490.00
cer_inicial = 22.54
cer_hoy = 632
tasa_anual_cupon = 0.02

# Fecha en la que estamos "comprando" el bono.
fecha_hoy = date(2025, 9, 27)

# --- 2. CALCULAR LA INVERSIÓN INICIAL EN TÉRMINOS REALES ---
# Para obtener la TIR real (TIR+CER), debemos "quitarle" la inflación acumulada
# al precio que pagamos hoy. Esto nos da el valor de nuestra inversión en "moneda original".

ajuste_cer = cer_hoy / cer_inicial
inversion_real = precio_bono_sucio / ajuste_cer

print(f"Precio pagado (Sucio): ${precio_bono_sucio:.2f}")
print(f"Coeficiente CER acumulado: {ajuste_cer:.4f}")
print(f"Inversión Inicial en Términos Reales: ${inversion_real:.2f}\n")


# --- 3. DEFINIR LOS FLUJOS DE FONDOS FUTUROS (en Términos Reales) ---
# Estos son los pagos que el bono todavía tiene que hacer, expresados
# en su valor nominal original (sin ajustar por CER).

# Flujo 1: 9 de Noviembre de 2025
fecha_flujo_1 = date(2025, 11, 9)
valor_flujo_1 = 20.0 + (60 * (tasa_anual_cupon / 2)) # 20.6

# Flujo 2: 9 de Mayo de 2026
fecha_flujo_2 = date(2026, 5, 9)
valor_flujo_2 = 20.0 + (40 * (tasa_anual_cupon / 2)) # 20.4

# Flujo 3: 9 de Noviembre de 2026 (Último pago)
fecha_flujo_3 = date(2026, 11, 9)
valor_flujo_3 = 20.0 + (20 * (tasa_anual_cupon / 2)) # 20.2


# --- 4. PREPARAR LOS DATOS PARA LA FUNCIÓN XIRR ---
# La función XIRR necesita una lista de fechas y una lista de valores.
# El primer valor es la inversión (siempre negativo).
fechas_flujos = [fecha_hoy, fecha_flujo_1, fecha_flujo_2, fecha_flujo_3]
valores_flujos = [-inversion_real, valor_flujo_1, valor_flujo_2, valor_flujo_3]

print("Flujos de fondos para el cálculo de la TIR Real:")
for fecha, valor in zip(fechas_flujos, valores_flujos):
    print(f"Fecha: {fecha}, Valor: {valor:.2f}")

# --- 5. CALCULAR Y MOSTRAR LA TIR ---
# La función xirr de pyxirr nos devuelve directamente la tasa anualizada.
try:
    # NOTA: El orden es fechas, luego valores.
    tir_real_anual = pyxirr.xirr(fechas_flujos, valores_flujos)
    
    print("\n-------------------------------------------")
    # Multiplicamos por 100 para mostrarlo como porcentaje
    print(f"TIR Real Anual Calculada: {tir_real_anual * 100:.2f}%")
    print("El rendimiento del bono es: TIR + CER")
    print(f"Rendimiento estimado: {tir_real_anual * 100:.2f}% + CER")
    print("-------------------------------------------")

except Exception as e:
    print(f"\nNo se pudo calcular la TIR. Error: {e}")

Precio pagado (Sucio): $1490.00
Coeficiente CER acumulado: 28.0390
Inversión Inicial en Términos Reales: $53.14

Flujos de fondos para el cálculo de la TIR Real:
Fecha: 2025-09-27, Valor: -53.14
Fecha: 2025-11-09, Valor: 20.60
Fecha: 2026-05-09, Valor: 20.40
Fecha: 2026-11-09, Valor: 20.20

-------------------------------------------
TIR Real Anual Calculada: 26.87%
El rendimiento del bono es: TIR + CER
Rendimiento estimado: 26.87% + CER
-------------------------------------------


In [15]:
# Calculando la TIR con numpy

import numpy as np
from datetime import date
from scipy.optimize import newton # <-- El buscador de raíces

# --- 1. DATOS DE ENTRADA (Igual que antes) ---
precio_bono_sucio = 1490
cer_inicial = 22.5440
cer_hoy = cer_hoy
tasa_anual_cupon = 0.02

# --- 2. PREPARAR LOS FLUJOS Y FECHAS (Igual que antes) ---
ajuste_cer = cer_hoy / cer_inicial
inversion_real = precio_bono_sucio / ajuste_cer

# Listas de fechas y valores
fechas_flujos = [
    date(2025, 9, 27),
    date(2025, 11, 9),
    date(2026, 5, 9),
    date(2026, 11, 9)
]
valores_flujos = [
    -inversion_real,
    20.0 + (60 * (tasa_anual_cupon / 2)),
    20.0 + (40 * (tasa_anual_cupon / 2)),
    20.0 + (20 * (tasa_anual_cupon / 2))
]

# Convertimos a arrays de NumPy para cálculos eficientes
fechas_np = np.array(fechas_flujos)
valores_np = np.array(valores_flujos)


# --- 3. DEFINIR LA FUNCIÓN DEL VALOR PRESENTE NETO (VPN) ---
# Esta es la función que queremos que valga cero.
# Toma la 'tasa' como argumento principal.

def vpn_xirr(tasa, fechas, valores):
    """
    Calcula el Valor Presente Neto para flujos en fechas irregulares.
    """
    # La primera fecha es nuestra referencia (t=0)
    fecha_inicial = fechas[0]
    
    # Calculamos los años transcurridos desde la fecha inicial para cada flujo
    dias_transcurridos = np.array([(fecha - fecha_inicial).days for fecha in fechas])
    anios_transcurridos = dias_transcurridos / 365.0
    
    # Calculamos el valor presente de cada flujo y los sumamos
    return np.sum(valores / ((1 + tasa) ** anios_transcurridos))


# --- 4. USAR EL BUSCADOR DE RAÍCES PARA ENCONTRAR LA TIR ---
# Le pedimos a 'newton' que encuentre la raíz de nuestra función 'vpn_xirr'.
# Empezamos con una estimación inicial de 0.1 (10%).
# 'args' son los argumentos adicionales que necesita nuestra función.

try:
    tir_real_anual = newton(vpn_xirr, 0.1, args=(fechas_np, valores_np))

    print("\n--- Cálculo 'Manual' con NumPy y SciPy ---")
    print("-------------------------------------------")
    print(f"TIR Real Anual Calculada: {tir_real_anual * 100:.2f}%")
    print(f"Rendimiento estimado: {tir_real_anual * 100:.2f}% + CER")
    print("-------------------------------------------")

except RuntimeError:
    print("\nEl buscador de raíces no pudo converger. Prueba con otra estimación inicial.")


--- Cálculo 'Manual' con NumPy y SciPy ---
-------------------------------------------
TIR Real Anual Calculada: 26.83%
Rendimiento estimado: 26.83% + CER
-------------------------------------------
